In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from openai import OpenAI

load_dotenv()

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
MODEL_NAME = "gpt-4o-mini"


In [2]:
import pandas as pd

df = pd.read_excel("サンプルデータ.xlsx", sheet_name="売上データ")

print(df.head())
print("行数:", len(df))
print("列名:", list(df.columns))


  カテゴリー  商品コード   商品名        売上日    単価   数量    原価
0    食品   1001   りんご 2023-01-01   200   50   120
1    食品   1002   バナナ 2023-01-01   150  100    80
2    食品   1003    牛乳 2023-01-02   180   80   100
3    衣服   2001  Tシャツ 2023-01-02  1500   20   800
4    衣服   2002  ジーンズ 2023-01-03  5000   10  2500
行数: 240
列名: ['カテゴリー', '商品コード', '商品名', '売上日', '単価', '数量', '原価']


In [3]:
# 商品ごとに売上個数（数量）を合計
product_qty = (
    df.groupby("商品名", as_index=False)["数量"]
      .sum()
      .sort_values("数量", ascending=False)
)

product_qty.head(10)


,商品名,数量
184,ヨーグルト,340
136,ビスケット,200
222,牛乳,180
114,バスキャップ,150
169,ポテトチップス,150
89,チョコレート,150
224,綿棒,150
161,ペーパーナプキン,150
30,キウイ,150
126,パイナップル,130


In [4]:
# 売上金額を計算
df["売上金額"] = df["単価"] * df["数量"]

# 商品ごとに 数量・売上金額・単価（平均） を集計
product_summary = (
    df.groupby("商品名", as_index=False)
      .agg(
          数量合計=("数量", "sum"),
          売上金額合計=("売上金額", "sum"),
          平均単価=("単価", "mean")
      )
      .sort_values("数量合計", ascending=False)
)

product_summary.head(10)




,商品名,数量合計,売上金額合計,平均単価
184,ヨーグルト,340,40800,120.0
136,ビスケット,200,16000,80.0
222,牛乳,180,32400,180.0
114,バスキャップ,150,15000,100.0
169,ポテトチップス,150,15000,100.0
89,チョコレート,150,15000,100.0
224,綿棒,150,15000,100.0
161,ペーパーナプキン,150,15000,100.0
30,キウイ,150,15000,100.0
126,パイナップル,130,39000,300.0


In [5]:
product_summary = product_summary[
    ["商品名", "数量合計", "平均単価", "売上金額合計"]
]

product_summary.head(10)


,商品名,数量合計,平均単価,売上金額合計
184,ヨーグルト,340,120.0,40800
136,ビスケット,200,80.0,16000
222,牛乳,180,180.0,32400
114,バスキャップ,150,100.0,15000
169,ポテトチップス,150,100.0,15000
89,チョコレート,150,100.0,15000
224,綿棒,150,100.0,15000
161,ペーパーナプキン,150,100.0,15000
30,キウイ,150,100.0,15000
126,パイナップル,130,300.0,39000


In [6]:
top10_text = product_summary.head(10).to_string(index=False)
print(top10_text)


     商品名  数量合計  平均単価  売上金額合計
   ヨーグルト   340 120.0   40800
   ビスケット   200  80.0   16000
      牛乳   180 180.0   32400
  バスキャップ   150 100.0   15000
 ポテトチップス   150 100.0   15000
  チョコレート   150 100.0   15000
      綿棒   150 100.0   15000
ペーパーナプキン   150 100.0   15000
     キウイ   150 100.0   15000
  パイナップル   130 300.0   39000


In [8]:
role = "あなたは小売業のデータ分析に強いマーケティング担当者です。"

prompt_text = f"""
以下は『売上個数（数量）』が多い順の人気商品ランキング（上位10）です。
この表から読み取れる傾向を3つ、改善・施策案を2つ、箇条書きで日本語で答えてください。
また「数量は多いが売上金額が伸びにくい可能性がある商品」もあれば指摘してください。

ランキング表:
{top10_text}
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
    temperature=0,
)

llm_comment = response.choices[0].message.content.strip()
print(llm_comment)


### 読み取れる傾向
1. **低価格商品の人気**: ヨーグルトやビスケットなど、比較的低価格の商品が上位にランクインしており、消費者の手に取りやすい傾向が見られる。
2. **数量と売上金額の乖離**: ヨーグルトは数量が最も多いが、平均単価が低いため、売上金額はそれほど高くない。数量が多い商品は必ずしも高い売上金額を生むわけではない。
3. **高単価商品の存在**: パイナップルのように、数量は少ないが平均単価が高い商品も存在し、売上金額に貢献している。

### 改善・施策案
1. **プロモーション戦略の見直し**: 数量が多いが売上金額が伸びにくい商品（例：ヨーグルト、ビスケット）に対して、セット販売や割引キャンペーンを実施し、単価を上げる施策を検討する。
2. **高単価商品のプロモーション強化**: パイナップルや牛乳などの高単価商品に対して、特別なプロモーションや試食イベントを行い、消費者の関心を引き、売上を伸ばす施策を実施する。

### 数量は多いが売上金額が伸びにくい可能性がある商品
- **ヨーグルト**: 数量は340と多いが、平均単価が120.0であるため、売上金額は40800とそれほど高くない。
- **ビスケット**: 数量は200だが、平均単価が80.0で、売上金額は16000と低い。


In [9]:
# LLMコメントをDataFrameに変換
llm_df = pd.DataFrame(
    llm_comment.split("\n"),
    columns=["LLM分析結果"]
)

# Excelに保存（複数シート）
with pd.ExcelWriter("商品人気ランキング_分析結果.xlsx", engine="openpyxl") as writer:
    product_summary.to_excel(writer, sheet_name="商品人気ランキング", index=False)
    llm_df.to_excel(writer, sheet_name="LLM分析コメント", index=False)

print("Excelファイルに保存しました。")


Excelファイルに保存しました。


In [10]:
import os
os.startfile("商品人気ランキング_分析結果.xlsx")
